In [1]:
import pandas as pd
import numpy as np

In [2]:
airport = pd.read_csv("Science.csv")

In [3]:
airport = airport.sort_values(by=["Year","Month"])

In [4]:
airport = airport.dropna()

In [5]:
data = airport.copy()

In [6]:
data = data.drop(columns=["Year"])

In [7]:
data.head(10)

,Passengers,Month
3,1575.27,0
7,1956.15,1
11,2365.56,2
1,1487.53,3
0,1235.75,4
6,1587.49,5
5,1875.66,6
2,1563.96,7
4,1762.00,8
10,2066.34,9


In [8]:
def get_diff(data):
    data['diff'] = data.Passengers.diff()    
    data = data.dropna()      
    return data
stationary_df = get_diff(data)

In [9]:
stationary_df.head(10)

,Passengers,Month,diff
7,1956.15,1,380.88
11,2365.56,2,409.41
1,1487.53,3,-878.03
0,1235.75,4,-251.78
6,1587.49,5,351.74
5,1875.66,6,288.17
2,1563.96,7,-311.70
4,1762.00,8,198.04
10,2066.34,9,304.34
9,2123.36,10,57.02


In [17]:

def time_plot(data, x_col, y_col, title):
    fig, ax = plt.subplots(figsize=(15,5))
    sns.lineplot(x_col, y_col, data=data, ax=ax, color='mediumblue', label='Total Passengers')
    
    second = data.groupby(data.Month.dt.year)[y_col].mean().reset_index()
    second.date = pd.to_datetime(second.date, format='%Y')
    sns.lineplot((second.date + datetime.timedelta(6*365/12)), y_col, data=second, ax=ax, color='red', label='Mean Passengers')   
    
    ax.set(xlabel = "Month",
           ylabel = "Passengers",
           title = title)
    
    sns.despine()

In [11]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [ ]:
time_plot(data, 'Month', 'Passengers', 'Monthly flights Before Diff Transformation')

In [20]:
def generate_supervised(data):
    supervised_df = data.copy()
    
    #create column for each lag
    for i in range(1,13):
        col = 'lag_' + str(i)
        supervised_df[col] = supervised_df['diff'].shift(i)
    
    #drop null values
    supervised_df = supervised_df.dropna().reset_index(drop=True)
    supervised_df.to_csv('model_df.csv', index=False)
    
    return supervised_df
model_df = generate_supervised(stationary_df)

In [24]:
model_df

,Passengers,Month,diff,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12
0,2268.75,1,389.41,-210.46,-33.56,57.02,304.34,198.04,-311.70,288.17,351.74,-251.78,-878.03,409.41,380.88
1,2502.84,2,234.09,389.41,-210.46,-33.56,57.02,304.34,198.04,-311.70,288.17,351.74,-251.78,-878.03,409.41
2,1689.85,3,-812.99,234.09,389.41,-210.46,-33.56,57.02,304.34,198.04,-311.70,288.17,351.74,-251.78,-878.03
3,1301.87,4,-387.98,-812.99,234.09,389.41,-210.46,-33.56,57.02,304.34,198.04,-311.70,288.17,351.74,-251.78
4,2126.73,5,824.86,-387.98,-812.99,234.09,389.41,-210.46,-33.56,57.02,304.34,198.04,-311.70,288.17,351.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,4936.49,8,-621.96,331.56,-266.52,1771.13,-1500.00,-287.42,-219.83,207.12,107.45,205.37,-471.79,1238.65,-268.25
128,4542.83,9,-393.66,-621.96,331.56,-266.52,1771.13,-1500.00,-287.42,-219.83,207.12,107.45,205.37,-471.79,1238.65
129,5247.11,10,704.28,-393.66,-621.96,331.56,-266.52,1771.13,-1500.00,-287.42,-219.83,207.12,107.45,205.37,-471.79
130,5893.07,11,645.96,704.28,-393.66,-621.96,331.56,-266.52,1771.13,-1500.00,-287.42,-219.83,207.12,107.45,205.37


In [25]:
model_df = model_df.iloc[:131]

In [26]:
model_df

,Passengers,Month,diff,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12
0,2268.75,1,389.41,-210.46,-33.56,57.02,304.34,198.04,-311.70,288.17,351.74,-251.78,-878.03,409.41,380.88
1,2502.84,2,234.09,389.41,-210.46,-33.56,57.02,304.34,198.04,-311.70,288.17,351.74,-251.78,-878.03,409.41
2,1689.85,3,-812.99,234.09,389.41,-210.46,-33.56,57.02,304.34,198.04,-311.70,288.17,351.74,-251.78,-878.03
3,1301.87,4,-387.98,-812.99,234.09,389.41,-210.46,-33.56,57.02,304.34,198.04,-311.70,288.17,351.74,-251.78
4,2126.73,5,824.86,-387.98,-812.99,234.09,389.41,-210.46,-33.56,57.02,304.34,198.04,-311.70,288.17,351.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,5558.45,7,331.56,-266.52,1771.13,-1500.00,-287.42,-219.83,207.12,107.45,205.37,-471.79,1238.65,-268.25,22.69
127,4936.49,8,-621.96,331.56,-266.52,1771.13,-1500.00,-287.42,-219.83,207.12,107.45,205.37,-471.79,1238.65,-268.25
128,4542.83,9,-393.66,-621.96,331.56,-266.52,1771.13,-1500.00,-287.42,-219.83,207.12,107.45,205.37,-471.79,1238.65
129,5247.11,10,704.28,-393.66,-621.96,331.56,-266.52,1771.13,-1500.00,-287.42,-219.83,207.12,107.45,205.37,-471.79


In [ ]:
def regressive_model(train_data, test_data, model, model_name):
    
    # Call helper functions to create X & y and scale data
    X_train, y_train, X_test, y_test, scaler_object = 
        scale_data(train_data, test_data)
    
    # Run regression model
    mod = model
    mod.fit(X_train, y_train)
    predictions = mod.predict(X_test)
    # Call helper functions to undo scaling & create prediction df
    original_df = load_data('../data/monthly_data.csv')
    unscaled = undo_scaling(predictions, X_test, scaler_object)
    unscaled_df = predict_df(unscaled, original_df)
    # Call helper functions to print scores and plot results
    get_scores(unscaled_df, original_df, model_name)
    plot_results(unscaled_df, original_df, model_name)